# Swap LRNA
Agent $\alpha$ swaps asset $i$ with the protocol for LRNA.

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath(os.path.join(os.path.abspath(''), '..')))

import inspect
from model.amm.omnipool_amm import swap_lrna, swap_lrna_delta_Qi, swap_lrna_delta_Ri

## Constraints

The swap will keep the swap invariant for asset $i$, $R_iQ_i$, constant.

## Requirements

#### Case 1: Asset $i$ sold
$$
\Delta r^\alpha \leq r_i^\alpha
$$
#### Case 2: LRNA sold
$$
-\Delta q^\alpha \leq q^\alpha
$$

## Updating AMM state

### Case 1: LRNA sold, $\Delta q^\alpha < 0$ specified

If $-\Delta q^\alpha > q^\alpha$, the user does not have enough LRNA to sell, and the transaction must fail.

$$
\begin{align}
\Delta Q_i &= -\Delta q^\alpha\\
\Delta R_i &= R_i\frac{- \Delta Q_i}{Q_i + \Delta Q_i}(1 - f_A)\\
\Delta L &= -\Delta Q_i\left(1 + (1 - f_A)\frac{Q_i}{Q_i + \Delta Q_i}\right)\\
\Delta r_i^\alpha &= - \Delta R_i\\
\end{align}
$$

In [2]:
print(inspect.getsource(swap_lrna_delta_Ri))

def swap_lrna_delta_Ri(old_state: dict, delta_Qi: float, i: int) -> float:
    return old_state['R'][i] * (- delta_Qi / (old_state['Q'][i] + delta_Qi))



### Case 2: Asset $i$ bought, $\Delta r_i^\alpha > 0$ specified

$$
\begin{align}
\Delta R_i &= -\Delta r_i^\alpha\\
\Delta Q_i &= Q_i\frac{-\Delta R_i}{R_i(1 - f_A) + \Delta R_i}\\
\Delta L &= -\Delta Q_i\left(1 + (1 - f_A)\frac{Q_i}{Q_i + \Delta Q_i}\right)\\
\Delta q^\alpha &= - \Delta Q_i\\
\end{align}
$$

If $-\Delta q^\alpha > q^\alpha$, the user does not have enough LRNA to sell, and the transaction must fail.

In [3]:
print(inspect.getsource(swap_lrna_delta_Qi))

def swap_lrna_delta_Qi(old_state: dict, delta_Ri: float, i: int) -> float:
    return old_state['Q'][i] * (- delta_Ri / (old_state['R'][i] + delta_Ri))



In [4]:
print(inspect.getsource(swap_lrna))

def swap_lrna(
        old_state: dict,
        old_agents: dict,
        trader_id: string,
        delta_Ra: float,
        delta_Qa: float,
        i: int,
        fee_assets: float = 0,
        fee_lrna: float = 0
) -> tuple:
    """Compute new state after LRNA swap"""

    if delta_Ra >= old_state['R'][i] * (1 - fee_assets):
        # insufficient assets in pool, transaction fails
        return old_state, old_agents

    if -delta_Qa > old_agents['q']:
        # agent doesn't have enough lrna
        return old_state, old_agents

    new_state = copy.deepcopy(old_state)
    new_agents = copy.deepcopy(old_agents)

    if delta_Qa < 0:
        delta_Q = -delta_Qa
        delta_R = old_state['R'][i] * -delta_Q / (delta_Q + old_state['Q'][i]) * (1 - fee_assets)
        delta_L = -delta_Q * (1 + (1 - fee_assets) * old_state['Q'][i] / (old_state['Q'][i] + delta_Q))
        delta_Ra = -delta_R
    elif delta_Ra > 0:
        delta_R = -delta_Ra
        delta_Q = old_state['Q'][i] * -delt